In [1]:
# Step a: Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Input, Dense
from keras import regularizers
from keras.callbacks import EarlyStopping

In [3]:
# Step b: Load the dataset
data = pd.read_csv("creditcard.csv")
data['Amount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1, 1))
data = data.drop(['Time'], axis=1)

In [4]:
# Train-test split
X_train, X_test = train_test_split(data, test_size=0.2, random_state=42)
X_train = X_train[X_train.Class == 0].drop(['Class'], axis=1).values
y_test = X_test['Class']
X_test = X_test.drop(['Class'], axis=1).values

In [5]:
# Step c & d: Define the Autoencoder model
input_dim = X_train.shape[1]
encoding_dim = 14
input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation="tanh", activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoder = Dense(int(encoding_dim / 2), activation="relu")(encoder)
decoder = Dense(int(encoding_dim / 2), activation='tanh')(encoder)
decoder = Dense(input_dim, activation='relu')(decoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)

In [6]:
# Step e: Compile and train the model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
autoencoder.fit(X_train, X_train, epochs=20, batch_size=32, validation_data=(X_test, X_test), callbacks=[early_stop], verbose=1)


Epoch 1/20
7108/7108 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step - loss: 0.8505 - val_loss: 0.7815
Epoch 2/20
7108/7108 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - loss: 0.7317 - val_loss: 0.7715
Epoch 3/20
7108/7108 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - loss: 0.7417 - val_loss: 0.7663
Epoch 4/20
7108/7108 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - loss: 0.7229 - val_loss: 0.7657
Epoch 5/20
7108/7108 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - loss: 0.7286 - val_loss: 0.7600
Epoch 6/20
7108/7108 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - loss: 0.7279 - val_loss: 0.7586
Epoch 7/20
7108/7108 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - loss: 0.7322 - val_loss: 0.7535
Epoch 8/20
7108/7108 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - loss: 0.7201 - val_loss: 0.7538
Epoch 9/20
7108/7108 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - loss: 0.7191 - val_loss: 0.7521
Epoch 10/20
7108/7108 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - loss: 0.7147 - val_loss: 0.7503
Epoch 11/20
7108/7108 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - loss: 0.7094 - val_loss: 0.7511
Epoch 12/20
7108/71

In [7]:
# Evaluate the model
predictions = autoencoder.predict(X_test)
mse = np.mean(np.power(X_test - predictions, 2), axis=1)
threshold = 50  # Set based on analysis
y_pred = [1 if e > threshold else 0 for e in mse]

1781/1781 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step


In [8]:
# Print metrics
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix


In [9]:
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.9983673326077034


In [10]:
print("Recall:", recall_score(y_test, y_pred))


Recall: 0.25510204081632654


In [11]:
print("Precision:", precision_score(y_test, y_pred))


Precision: 0.5555555555555556


In [12]:
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Confusion Matrix:
 [[56844    20]
 [   73    25]]


In [ ]:
Use Autoencoder to implement anomaly detection. Build the model by using:
a. Import required libraries
b. Upload / access the dataset
c. Encoder converts it into latent representation
d. Decoder networks convert it back to the original input
e. Compile the models with Optimizer, Loss, and Evaluation Metrics